In [ ]:
! pip install -q -U langchain-openai
! pip install -q -U langchain-community

In [ ]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_openai import ChatOpenAI
import pandas as pd

In [ ]:
data = pd.read_csv("../data/train.csv")
display(data.head())

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스


In [ ]:
import random

def create_examples_from_dataframe(df, num_examples):
    examples = []
    for _, row in df.sample(n=num_examples).iterrows():
        example = {
            "dialogue": row["dialogue"],
            "summary": row["summary"],
        }
        examples.append(example)
    return examples

In [ ]:
# 데이터프레임에서 examples 생성
examples = create_examples_from_dataframe(data, num_examples=len(data))
examples

[{'dialogue': '#Person1#: 아, 좋은 아침입니다. 로빈슨 씨이시죠?\r\n#Person2#: 네.\r\n#Person1#: 자리에 앉으세요.\r\n#Person2#: 감사합니다.\r\n#Person1#: 네. 지원서를 받았습니다. 알다시피, 저희 회사에 지원할 때는 학력 및 최근의 근무 경력에 대해 알아야 합니다.\r\n#Person2#: 알겠습니다.\r\n#Person1#: 먼저, A-level은요?\r\n#Person2#: 네, 세 개 있습니다. 지리, 수학, 물리입니다.\r\n#Person1#: 지리, 수학, 물리. 알겠습니다. 그럼 학위는 어떻게 되나요?\r\n#Person2#: 맨체스터 대학에 다녔고, 전공은 수자원 관리로 공학 학위를 받았습니다.\r\n#Person1#: 아하, 알겠습니다.\r\n#Person2#: 그리고 업무 경험은 1996년에 졸업 후 인도에서 시작했습니다. 인도 정부에서 일했습니다.\r\n#Person1#: 자원봉사로 일하셨나요?\r\n#Person2#: 아니요, 3년간의 물 관개 프로젝트였습니다.\r\n#Person1#: 그것은 흥미롭군요. 어떻게 그것을 조직했나요? 그것은 영국 회사가 아니었으니까요.\r\n#Person2#: 아니요, 알고 있습니다. 저의 대학은 인도의 공과 대학과 연계가 있었습니다. 그래서 그 수준에서 조직되었습니다.\r\n#Person1#: 그 후에는요?\r\n#Person2#: 그 후에는 돌아와서 쉐필드로 이사하고, 그 후로는 래티머 엔지니어링에서 일하고 있습니다.\r\n#Person1#: 래티머에서 정확히 어떤 일을 하고 있나요?\r\n#Person2#: 아, 물 관개 일에 다시 참여하고 있습니다. 이번에는 프로젝트 연구 조수로 일하고 있습니다.\r\n#Person1#: 좋습니다. 정보를 받았습니다. 이제 저희가 여기서 찾고 있는 것에 대한 것보다 일반적인 논의로 넘어가겠습니다...',
  'summary': '#Person1#은 로빈슨 씨를 면접하고 그의 학력 및 최근의 업무 경력

In [ ]:
# 4개의 예시를 임의로 추출
few_shot_examples = random.sample(examples, 4)
few_shot_examples

[{'dialogue': '#Person1#: 델라, 이 사람이 조니야.\n#Person2#: 오, 안녕 조니.\n#Person1#: 안녕. 이번 수요일이 제럴드의 생일이야. 토니와 에이미가 그에게 파티를 열자고 제안했어. 너도 올래?\n#Person2#: 이번 수요일? 어떤 종류의 파티야?\n#Person1#: 변장 파티야.\n#Person2#: 재미있겠네. 어디서 열리니?\n#Person1#: 내 집에서, 우리는 내일 오후에 집을 꾸밀 예정이야.\n#Person2#: 제럴드는 파티에 대해 알고 있나요?\n#Person1#: 아니, 우리는 그를 놀래키고 싶어.\n#Person2#: 나도 가고 싶지만, 일에 치여 있어. 저녁 시간을 내는 것이 가능할지 모르겠어.\n#Person1#: 벨라야, 제럴드를 좋아한다는 것을 알고 있어, 그리고 제럴드도 너를 높이 평가해. 하지만 너는 항상 바빠서, 그는 너에게 데이트를 제안할 기회가 없었어. 이번 기회를 주는 것도 좋을 것 같아.\n#Person2#: 그럼, 나는 그의 생일 파티 서프라이즈인가?\n#Person1#: 그렇게 말할 수 있겠네.\n#Person2#: 알았어, 나도 갈게. 오늘 저녁에 그에게 선물을 사야겠네.',
  'summary': '조니는 델라에게 전화를 걸어 토니와 에이미와 함께 제럴드에게 깜짝 파티를 열어달라고 요청한다. 델라는 처음에는 일 때문에 망설이지만, 그 후에 동의한다.'},
 {'dialogue': '#Person1#: 오늘 날씨가 참 좋지 않나요?\n#Person2#: 네, 그렇습니다. 폭우 후에 이렇게 좋은 날씨는 정말 좋네요.\n#Person1#: 다시 태양을 볼 수 있어서 좋네요! 공기도 상쾌하고요.\n#Person2#: 이렇게 좋은 날씨가 계속 이어졌으면 좋겠어요.\n#Person1#: 이 시기의 날씨는 정말 예측하기 힘들죠.\n#Person2#: 맞아요. 내일 날씨가 어떻게 될지 모르잖아요.\n#Person1#: 봐요! 비가 올 것 같아요.\n#Person2#: 기상 예보에 따

In [ ]:
# 예시 템플릿 정의
example_template = "[대화]\n{dialogue}\n\n[요약]\n{summary}"

# 예시 프롬프트 템플릿 생성
example_prompt = PromptTemplate(
    input_variables=["dialogue", "summary"],
    template=example_template
)

# Few-shot 프롬프트 템플릿 정의
few_shot_prompt = FewShotPromptTemplate(
    examples=few_shot_examples,  # 2개의 예시 사용
    example_prompt=example_prompt,
    prefix="당신은 대화를 요약해주는 유능한 AI입니다. \
당신의 임무는 다음에 나오는 대화를 요약하는 것입니다. \
당신의 대답은 오직 제공된 대화에만 근거해야 합니다. \
요약 스타일을 유지해야만 합니다.",
    suffix="[대화]\n{dialogue}\n\n[요약]\n ",
    input_variables=["dialogue"]
)

In [ ]:
test = pd.read_csv('../data/test.csv')
test.head()

,fname,dialogue
0,test_0,"#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. \n#Person2#: 네, ..."
1,test_1,#Person1#: 드디어 왔네! 왜 그렇게 오래 걸렸어?\n#Person2#: 또...
2,test_2,"#Person1#: 케이트, 무슨 일이 일어났는지 너는 믿지 못할거야. \n#Per..."
3,test_3,"#Person1#: 생일 축하해, 이건 너를 위한 거야, 브라이언.\n#Person..."
4,test_4,#Person1#: 이 올림픽 공원이 정말 크네요!\n#Person2#: 네. 지금...


In [ ]:
# 새로운 대화 데이터
new_dialogue = test['dialogue'][0]
# Few-shot 프롬프트 생성
prompt = few_shot_prompt.format(dialogue=new_dialogue)

print(prompt)

당신은 대화를 요약해주는 유능한 AI입니다. 당신의 임무는 다음에 나오는 대화를 요약하는 것입니다. 당신의 대답은 오직 제공된 대화에만 근거해야 합니다. 요약 스타일을 유지해야만 합니다.

[대화]
#Person1#: 델라, 이 사람이 조니야.
#Person2#: 오, 안녕 조니.
#Person1#: 안녕. 이번 수요일이 제럴드의 생일이야. 토니와 에이미가 그에게 파티를 열자고 제안했어. 너도 올래?
#Person2#: 이번 수요일? 어떤 종류의 파티야?
#Person1#: 변장 파티야.
#Person2#: 재미있겠네. 어디서 열리니?
#Person1#: 내 집에서, 우리는 내일 오후에 집을 꾸밀 예정이야.
#Person2#: 제럴드는 파티에 대해 알고 있나요?
#Person1#: 아니, 우리는 그를 놀래키고 싶어.
#Person2#: 나도 가고 싶지만, 일에 치여 있어. 저녁 시간을 내는 것이 가능할지 모르겠어.
#Person1#: 벨라야, 제럴드를 좋아한다는 것을 알고 있어, 그리고 제럴드도 너를 높이 평가해. 하지만 너는 항상 바빠서, 그는 너에게 데이트를 제안할 기회가 없었어. 이번 기회를 주는 것도 좋을 것 같아.
#Person2#: 그럼, 나는 그의 생일 파티 서프라이즈인가?
#Person1#: 그렇게 말할 수 있겠네.
#Person2#: 알았어, 나도 갈게. 오늘 저녁에 그에게 선물을 사야겠네.

[요약]
조니는 델라에게 전화를 걸어 토니와 에이미와 함께 제럴드에게 깜짝 파티를 열어달라고 요청한다. 델라는 처음에는 일 때문에 망설이지만, 그 후에 동의한다.

[대화]
#Person1#: 오늘 날씨가 참 좋지 않나요?
#Person2#: 네, 그렇습니다. 폭우 후에 이렇게 좋은 날씨는 정말 좋네요.
#Person1#: 다시 태양을 볼 수 있어서 좋네요! 공기도 상쾌하고요.
#Person2#: 이렇게 좋은 날씨가 계속 이어졌으면 좋겠어요.
#Person1#: 이 시기의 날씨는 정말 예측하기 힘들죠.
#Person2#: 맞아요. 내일 날씨가 어떻게 될지

In [ ]:
llm = ChatOpenAI(model_name='gpt-4-turbo', openai_api_key="", temperature=0.3, max_tokens=512)

In [ ]:
prompt = few_shot_prompt.format(dialogue=test['dialogue'][0])
summary = llm.invoke(prompt)
print(summary.content.strip())

#Person1#는 #Person2#에게 회사 내부 및 외부 통신을 이메일과 공식 메모로 제한하고 즉시 메시지 사용을 금지하는 새로운 정책을 받아쓰게 하며, 이 정책을 모든 직원에게 오후 4시 전에 전달하도록 지시합니다.


In [ ]:
from tqdm.notebook import tqdm

summary_list = []
for dialouge in tqdm(test['dialogue']):
    # 임의로 4개의 few-shot 예제를 추출
    few_shot_examples = random.sample(examples, 4)

    # Few-shot 프롬프트 템플릿 정의
    few_shot_prompt = FewShotPromptTemplate(
    examples=few_shot_examples,  # 2개의 예시 사용
    example_prompt=example_prompt,
    prefix="당신은 대화를 요약해주는 유능한 AI입니다. \
당신의 임무는 다음에 나오는 대화를 요약하는 것입니다. \
당신의 대답은 오직 제공된 대화에만 근거해야 합니다. \
요약 스타일을 유지해야만 합니다.",
    suffix="[대화]\n{dialogue}\n\n[요약]\n ",
    input_variables=["dialogue"]
)
    # Few-shot 프롬프트 생성
    prompt = few_shot_prompt.format(dialogue=dialouge)

    # 언어 모델을 사용하여 요약 생성
    summary = llm.invoke(prompt)
    print("="*20, "[ 대화 ]", "="*20)
    print(dialouge)
    print("="*20, "[ 요약 ]", "="*20)
    print(summary.content.strip())
    print()

    summary_list.append(summary.content.strip())

  0%|          | 0/499 [00:00<?, ?it/s]

==================== [ 대화 ] ====================
#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.
#Person2#: 그게 다신가요?
#Person1#: 네. 이 메모를 오후 4시 전에 모든 직원에게 타이핑하여 배포해 주세요.
==================== [ 요약 ] ====================
더슨 씨는 실장님의 지시에 따라 내부 메모를 받아쓰고, 모든 사무실 통신을 이메일과 공식 메모로 제한하며 즉시 메시지 사용을 금지하는 새로운 정

In [ ]:
test['summary'] = summary_list

In [ ]:
print(test['dialogue'][0])
print()
print(test['summary'][0])
print()
print(test['dialogue'][1])
print()
print(test['summary'][1])

#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.
#Person2#: 그게 다신가요?
#Person1#: 네. 이 메모를 오후 4시 전에 모든 직원에게 타이핑하여 배포해 주세요.

더슨 씨는 실장님의 지시에 따라 내부 메모를 받아쓰고, 모든 사무실 통신을 이메일과 공식 메모로 제한하며 즉시 메시지 사용을 금지하는 새로운 정책을 전달한다. 이 정책은 내부와 외부 통신 모두에 적용되며, 위반 시 직무 정지 또는 해고 처분을 받게 된다.

#Person1#: 드디어 왔네! 왜 그렇게 오래 걸렸어?


In [ ]:
test[['fname', 'summary']].to_csv("submission_fewshot.csv", index=False)